# Weighted minhashing
## Don MacMillen

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

You will have better results viewing this notebook on the nbviewer hosted by the Jupyter project rather than on github.  Look at the following:

https://nbviewer.jupyter.org/github/macd/dbm_notebooks/blob/master/weighted_minhashing.ipynb

This is a notebook that implements and (eventually) compares the QoR and runtime of the weighted min hashing schemes of Ioffe [1] and Shrivastava [2].  It is very much a work in progress at the moment.

We first implement Ioffe's method and we start with some global state and scratch space (to avoid re-allocating) for the minhash algo

In [1]:
using Distributions


immutable WMinIHash
    # Pre-calculated random state
    master_seed::Tuple{Int, Int}
    r::Array{Float64, 2}
    lc::Array{Float64, 2}
    beta::Array{Float64, 2}
    
    # scratch space for minhash
    t::Vector{Int}
    la::Vector{Float64}
    ly::Vector{Float64}
end


function WMinIHash(nfeatures, numhash, master_seed)
    # So, as of 21 Aug 2016, the Distributions package only uses
    # Base.GLOBAL_RNG as its source of entropy. Bummer.
    srand(master_seed[1])
    gma = Gamma(2, 1)
    r = rand(gma, nfeatures, numhash)
    srand(master_seed[2])
    gmb = Gamma(2, 1)
    lc = log.(rand(gmb, nfeatures, numhash))
    beta = rand(nfeatures, numhash)
    t = zeros(Int, nfeatures)
    la = ones(nfeatures)
    ly = zeros(nfeatures)
    wmih = WMinIHash(master_seed, r, lc, beta, t, la, ly)
end

WMinIHash

Here is the heart of the Ioffe algo.  It follows the paper.

In [2]:
# mintuples is pre-allocated (but reinit still uses mem?)
function i_minhash!(x, w, mintuples)
    nf, nh = size(w.r)
    # What is best init for mintuples?
    mintuples[:] = (0, 0)
    for h = 1:nh
        w.t[:] = 0
        w.la[:] = typemax(eltype(w.la))   # because we take the min later
        w.ly[:] = 0.0
        for k in 1:nf
            x[k] == 0.0 && continue
            w.t[k] = floor(log(x[k]) / w.r[k, h] + w.beta[k, h])
            w.ly[k] = w.r[k, h] * (w.t[k] - w.beta[k, h])
            w.la[k] = w.lc[k, h] - w.ly[k] - w.r[k, h]
        end
        kstar = indmin(w.la)
        mintuples[h] = (kstar, Int(w.t[kstar]))
    end
    return mintuples
end

i_minhash! (generic function with 1 method)

Now we need the test scaffolding to exercise.  This needs more thought and more work.

In [3]:
function i_minhash_all!(fp, md, wmih)
    ndata, nfeatures = size(md)
    M = nfeatures
    _, nhashes = size(fp)
    mintuples = Vector{Tuple{Int, Int}}(nhashes)
    for i in 1:ndata
        fp[i, :] = i_minhash!(md[i, :], wmih, mintuples)
    end
    nothing
end


function syndata(nrows, ncols, scale=1.0)
    rng = MersenneTwister(1137)
    #srand(1137)
    #rng = Weibull()
    md = scale * rand(rng, nrows, ncols)
    return md
end

wsim(x, y) = sum(min.(x, y)) / sum(max.(x, y))

# Pick one vector to compare the others to
const COMPARE = 51


function test_ioffe(nhashes=256, ndata=500, nfeatures=128)
    m = ones(nfeatures)
    scale = 1.0
    md = syndata(ndata, nfeatures, scale)

    iwmh = WMinIHash(nfeatures, nhashes, (919, 1137))
    finger_prints = Matrix{Tuple{Int, Int}}(ndata, nhashes)
    i_minhash_all!(finger_prints, md, iwmh)    

    error = []
    for i in 1:ndata
        x = finger_prints[i, :]
        y = finger_prints[COMPARE, :]
        sim = sum(x .== y) / nhashes
        exact_sim = wsim(md[i, :], md[COMPARE, :])
        push!(error, ((sim - exact_sim)/exact_sim, i))
    end
    #for (err, i) in sort(error)
    #    println(err, "   ", i)
    #end
    # TODO: plot a histogram of the errors
    # TODO: plot max error vs. the number of hashes
    # TODO: vary the size, number, and sparsity of the data vectors
    println("max error: ", maximum([abs(x[1]) for x in error]))
end

test_ioffe()

max error: 0.1716871583805694


And now for Shrivastava's red / green weighted minhashing

In [4]:
import Base.Random: mt_setempty!, MTCacheLength

# Reseeding the rng's inside an inner loop is a bad idea with the
# current implementation of MersenneTwister.  This is because it
# calls dsfmt_init_by_array which does a _lot_ of processing.
# In many cases we want to reset the seeds to those already seen, 
# so we cache a copy of the rng and restore if we find it in the cache.

rng_cache = Dict{Int, MersenneTwister}()

function rsrand(r::MersenneTwister, n::Int)
    if haskey(rng_cache, n)
        copy!(r, rng_cache[n])
    else
        srand(r, n)
        rng_cache[n] = copy(r)
    end
    return r
end


type WMinHash
    master_seed::Int
    m::Vector{Float64}
    rng::MersenneTwister
    seeds::Vector{Int}
    M::Int
    int2compM::Vector{Tuple{Int, Int}}
end


function WMinHash(max_hashes::Int, master_seed::Int, m::Vector{Float64})
    rng = MersenneTwister(master_seed)
    seeds = [abs(x) for x in rand(rng, Int, max_hashes)]
    wmh = WMinHash(master_seed, copy(m), rng, seeds, ceil(sum(m)), calc_arraymap(m))
end

WMinHash

With that out of the way, we can implement the three functions that are in the paper.

In [5]:
function calc_arraymap(m)
    int2compM = Vector{Tuple{Int, Int}}()
    D = length(m)
    Mi = 0
    for i = 1:D
        mi = ceil(Int, m[i])
        for j = 1:mi
             push!(int2compM, (i, Mi))
        end
        Mi += mi
    end
    return int2compM
end


function rg_minhash!(x, wmh::WMinHash, hashes::Vector{Int})
    num_hashes = length(hashes)
    num_hashes > length(wmh.seeds) && error("too many hashes")
    # (re) initialize the hashes
    hashes[:] = 0
    @inbounds for i = 1:num_hashes
        rsrand(wmh.rng, wmh.seeds[i])
        while true
            r = wmh.M * rand(wmh.rng)::Float64
            isgreen(r, x, wmh) && break
            rsrand(wmh.rng, ceil(Int, 1e6 * r))
            hashes[i] += 1
        end
    end
    hashes
end


function isgreen(r, x, wmh)
    index = ceil(Int, r)
    i, Mi = wmh.int2compM[index]
    if r <= Mi + x[i]
        return true
    end
    return false
end

isgreen (generic function with 1 method)

And now for the test scaffolding for red / green minhashing

In [6]:
function w(i, j)
    return wsim(md[i, :], md[j, :])
end

function es(i, j)
    _, nfeatures = size(finger_prints)
    return sum(finger_prints[i,:] .== finger_prints[j,:]) / nfeatures
end

function rg_minhash_all!(fp, md, wmh)
    ndata, nfeatures = size(md)
    # assume each component of x is bounded by one. Not true in general
    M = nfeatures
    _, nhashes = size(fp)
    hashes = zeros(Int, nhashes)
    for i in 1:ndata
        fp[i, :] = rg_minhash!(md[i, :], wmh, hashes)
    end
    nothing
end

# Make these global for debugging at the repl
md = Matrix{Float64}()
finger_prints = Matrix{Float64}()

# if both x[i] and y[i] are zero, then use 1 in the denom instead of zero.
@inline function esim(x, y)
    return sum(1.0 - (abs.(x .- y) ./ max.(1, x, y))) / length(x)
end

function test_rg(nhashes=256, ndata=500, nfeatures=128)
    global md, finger_prints
    md = syndata(ndata, nfeatures)
    #md = weibulldata(ndata, nfeatures)
    
    m = ones(nfeatures)
    wmh = WMinHash(1000000, 1137, m)
    finger_prints = zeros(Int, ndata, nhashes)

    rg_minhash_all!(finger_prints, md, wmh)
    fp = copy(finger_prints[51, :])
    error = []
    for i in 1:ndata
        sim1 = sum(finger_prints[i, :] .== fp) / nhashes
        sim2 = esim(finger_prints[i, :], fp)
        exact_sim = wsim(md[i, :], md[51, :])
        push!(error, ((sim1 - exact_sim)/exact_sim, exact_sim, sim1, sim2, i))
    end
    #for (err, exs, s1, s2, i) in sort(error)
    #    println(err, "   ", exs, "   ", s1, "   ", s2, "   ", i)
    #end
    println("max error: ", maximum([abs(x[1]) for x in error]))
end

test_rg()

max error: 0.1612122653955883


In [7]:
@time test_rg(1000)

max error: 0.07574762798830523
  1.147462 seconds (2.13 M allocations: 123.812 MB, 5.43% gc time)


In [8]:
@time test_ioffe(1000)

max error: 0.07796156862226696
  1.905385 seconds (11.85 k allocations: 33.089 MB, 0.15% gc time)


OK, well, they both seem to be working, with the red / green hashing getting slightly smaller max errors for the same number of hashes.   It also looks like red green is about 1.7 times faster **but** I may have some memory / type issues with the r/g code (see the 2.13 M allocations).  Will need to debug later. **Need to test much more and get detailed timings**

### References

[1] S. Ioffe, "Improved consistent sampling, weighted minhash and L1 sketching", In ICDM, pp. 246-255, Sydney, AU, 2010

[2] A. Shrivastava, "Simple and Efficient Weighted Minwise Hashing", NIPS, Barcelona, 2016